# Create useful CSV-Files from the saved queries and responses to the LinkedinRecruiter Data

In [56]:
import pandas as pd
import sqlite3
import json

In [97]:
conn = sqlite3.connect('linkedin7.db')

## From-To-Number-Table

In [87]:
# Select country_from, country_to, count, time to export something useful:

query = """SELECT country_From, country_to, count as number_people_who_indicated, datetime(query_time, 'unixepoch', 'localtime') as "query_time",
datetime( round(query_time/(60*60*24*3),0)*(60*60*24*3), 'unixepoch', 'localtime') as "query_time_round"
FROM geofacets gf left outer join queries as qu on gf.fk_queries = qu.pk_queries
WHERE count is not null AND country_from in 
 (select country_to from geofacets group by country_to)
Group by country_from, country_to, query_time_round
Order by country_from, country_to"""
dffromto = pd.read_sql_query(query,conn)
dffromto.to_csv("LinkedInRecruiter_country_from_to_number.csv")
dffromto.head(10)

,country_from,country_to,number_people_who_indicated,query_time,query_time_round
0,Afghanistan,Central African Republic,4,2020-11-03 15:23:58,2020-11-04 01:00:00
1,Afghanistan,Central African Republic,4,2020-11-18 09:52:34,2020-11-19 01:00:00
2,Afghanistan,Djibouti,20,2020-07-24 04:45:26,2020-07-25 02:00:00
3,Afghanistan,Djibouti,14,2020-10-08 19:32:19,2020-10-08 02:00:00
4,Afghanistan,Djibouti,15,2020-10-21 04:36:36,2020-10-20 02:00:00
5,Afghanistan,Djibouti,16,2020-11-03 15:50:58,2020-11-04 01:00:00
6,Afghanistan,Djibouti,16,2020-11-18 10:19:44,2020-11-19 01:00:00
7,Afghanistan,Gaza Strip,1,2020-07-24 04:43:19,2020-07-25 02:00:00
8,Afghanistan,Gaza Strip,1,2020-10-08 19:30:10,2020-10-08 02:00:00
9,Afghanistan,Iran,25,2020-10-08 14:56:03,2020-10-08 02:00:00


## now merge more information into it: Number of Linkedin-users, GDP, GPS, ....

### LinkedIn users per country

In [88]:
# get the number of linkedin-users per country, group by 3 days so that it can be merged with a collection in the same timeframe

query = """SELECT region,max(count) linkedinusers, datetime(max(query_time), 'unixepoch', 'localtime') as "query_time", datetime( round(query_time/(60*60*24*3),0)*(60*60*24*3), 'unixepoch', 'localtime') as "query_time_round"
--,datetime(min(query_time), 'unixepoch', 'localtime') as "query_time_min"
FROM baserates 
WHERE count is not null
group by region, query_time_round
Order by region,query_time desc--n asc"""

dfnusers = pd.read_sql_query(query,conn)
dfnusers.to_csv("LinkedInRecruiter_country_number of linkedin-users.csv")
dfnusers.head(40)

,region,linkedinusers,query_time,query_time_round
0,Afghanistan,238605,2020-11-18 09:34:50,2020-11-19 01:00:00
1,Afghanistan,242303,2020-11-03 15:06:44,2020-11-04 01:00:00
2,Afghanistan,240736,2020-10-20 17:02:09,2020-10-20 02:00:00
3,Afghanistan,237865,2020-10-08 18:47:52,2020-10-08 02:00:00
4,Afghanistan,231293,2020-07-24 03:59:26,2020-07-25 02:00:00
5,Albania,296664,2020-11-18 09:31:34,2020-11-19 01:00:00
6,Albania,297900,2020-11-03 15:03:29,2020-11-04 01:00:00
7,Albania,296746,2020-10-20 16:58:52,2020-10-20 02:00:00
8,Albania,295648,2020-10-08 18:44:37,2020-10-08 02:00:00
9,Albania,289925,2020-07-24 03:56:13,2020-07-25 02:00:00


In [89]:
dffromtobase = dffromto.merge(dfnusers,left_on=["country_from","query_time_round"],right_on=["region","query_time_round"])
dffromtobase = dffromtobase.merge(dfnusers,left_on=["country_to","query_time_round"],right_on=["region","query_time_round"])
dffromtobase.rename(columns={"linkedinusers_x":"linkedinusers_from","linkedinusers_y":"linkedinusers_to"},inplace=True, errors="raise")
dffromtobase.drop(["query_time_x","query_time_y","query_time","region_x","region_y"], axis=1, inplace=True)
dffromtobase["normalized1"] = dffromtobase['number_people_who_indicated'] / (dffromtobase['linkedinusers_from'] + dffromtobase['linkedinusers_to'])
dffromtobase["normalized2"] = dffromtobase['number_people_who_indicated']**2 / (dffromtobase['linkedinusers_from'] * dffromtobase['linkedinusers_to'])
dffromtobase 

,country_from,country_to,number_people_who_indicated,query_time_round,linkedinusers_from,linkedinusers_to,normalized1,normalized2
0,Afghanistan,Central African Republic,4,2020-11-04 01:00:00,242303,21307,0.000015,3.099124e-09
1,Bangladesh,Central African Republic,6,2020-11-04 01:00:00,2293932,21307,0.000003,7.365456e-10
2,Belgium,Central African Republic,5,2020-11-04 01:00:00,3706951,21307,0.000001,3.165198e-10
3,Benin,Central African Republic,20,2020-11-04 01:00:00,168338,21307,0.000105,1.115207e-07
4,Burkina Faso,Central African Republic,20,2020-11-04 01:00:00,165056,21307,0.000107,1.137382e-07
...,...,...,...,...,...,...,...,...
27352,Switzerland,Austria,1130,2020-07-25 02:00:00,2632934,1124659,0.000301,4.312172e-07
27353,Turkey,Austria,1171,2020-07-25 02:00:00,7998723,1124659,0.000128,1.524306e-07
27354,United Kingdom,Austria,1972,2020-07-25 02:00:00,28478411,1124659,0.000067,1.214164e-07
27355,United States,Austria,1604,2020-07-25 02:00:00,168362319,1124659,0.000009,1.358761e-08


In [90]:
dffromtobase[(dffromtobase.country_from=="Germany") & (dffromtobase.number_people_who_indicated > 1000) ].head(40)

,country_from,country_to,number_people_who_indicated,query_time_round,linkedinusers_from,linkedinusers_to,normalized1,normalized2
5188,Germany,Turkey,1307,2020-07-25 02:00:00,10649480,7998723,0.000070,2.005405e-08
6260,Germany,Brazil,1145,2020-11-04 01:00:00,11122309,46450132,0.000020,2.537634e-09
6738,Germany,Brazil,1162,2020-11-19 01:00:00,11469490,44976651,0.000021,2.617466e-09
7150,Germany,Brazil,1139,2020-10-20 02:00:00,11551758,44753978,0.000020,2.509387e-09
7428,Germany,Portugal,1979,2020-10-20 02:00:00,11551758,3061711,0.000135,1.107336e-07
10288,Germany,Australia,4882,2020-07-25 02:00:00,10649480,11296386,0.000222,1.981197e-07
10360,Germany,Brazil,1111,2020-07-25 02:00:00,10649480,44515975,0.000020,2.603657e-09
10376,Germany,Canada,6031,2020-07-25 02:00:00,10649480,17317997,0.000216,1.972207e-07
10648,Germany,Ireland,2128,2020-07-25 02:00:00,10649480,1750105,0.000172,2.429689e-07
10689,Germany,Italy,4350,2020-07-25 02:00:00,10649480,13771863,0.000178,1.290201e-07


In [82]:
dffromtobase.query("country_from == 'Afghanistan'")#.tail(30)

,country_from,country_to,number_people_who_indicated,query_time_round,linkedinusers_from,linkedinusers_to,normalized1,normalized2
0,Afghanistan,Central African Republic,4,2020-11-04 01:00:00,242303,21307,0.000015,3.099124e-09
47,Afghanistan,Djibouti,16,2020-11-04 01:00:00,242303,30581,0.000059,3.454852e-08
88,Afghanistan,Iran,26,2020-11-04 01:00:00,242303,2962192,0.000008,9.418347e-10
122,Afghanistan,Iraq,275,2020-11-04 01:00:00,242303,625447,0.000317,4.990179e-07
155,Afghanistan,Kosovo,5,2020-11-04 01:00:00,242303,56694,0.000017,1.819886e-09
...,...,...,...,...,...,...,...,...
2233,Afghanistan,Syria,34,2020-10-20 02:00:00,240736,269448,0.000067,1.782140e-08
2274,Afghanistan,Tajikistan,36,2020-10-20 02:00:00,240736,39873,0.000128,1.350159e-07
2306,Afghanistan,Turkmenistan,5,2020-10-20 02:00:00,240736,23699,0.000019,4.381965e-09
2335,Afghanistan,Uzbekistan,27,2020-10-20 02:00:00,240736,179520,0.000064,1.686839e-08


In [73]:
#dffromtobase.drop(['count(*)_x', 'max(count)_x','min(count)_x','query_time_x','count(*)_y', 'max(count)_y','min(count)_y','query_time_y'], axis=1, inplace=True)
#dffromtobase["normalized"] = dffromtobase.count / (dffromtobase.nusers_x + dffromtobase.nusers_y)
#dffromtobase["normalized"] = dffromtobase['count'] / (dffromtobase['nusers_x'] + dffromtobase['nusers_y'])
dffromtobase["normalized1clipped"] = dffromtobase["normalized1"].clip(0,0.002) 
#dffromtobase

In [91]:
dffromtobase.to_csv("LinkedInRecruiter_dffromtobase_merged_5.csv")

### Add GDP-information

In [92]:
path = "N:/Theile/bibliometry/scopus_rp_2020/"
dfworld = pd.read_csv(path + "hdi-vs-gdp-per-capita.csv")
dfworld["year"] = pd.to_numeric( dfworld.Year,errors='coerce')
dfworld.columns = ["countryname","countrycode","Year","hihd","gdp","population","year"]
dfworld["countrycode"] = dfworld.countrycode.str.lower()
dfworld
dfworld.info()
dfworld["maxgdp"] = dfworld[dfworld.year>2000].groupby("countrycode")["gdp"].transform("max")
dfworld["maxhdi"] = dfworld[dfworld.year>2000].groupby("countrycode")["hihd"].transform("max")

dfworld1 = dfworld.loc[dfworld.year==2017].copy()
dfworld1.drop(["Year","hihd","gdp"],axis=1,inplace=True)
dfworld1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49569 entries, 0 to 49568
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   countryname  49569 non-null  object 
 1   countrycode  48545 non-null  object 
 2   Year         49569 non-null  object 
 3   hihd         3210 non-null   float64
 4   gdp          17712 non-null  float64
 5   population   46883 non-null  float64
 6   year         49499 non-null  float64
dtypes: float64(4), object(3)
memory usage: 2.6+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 242 entries, 217 to 49566
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   countryname  242 non-null    object 
 1   countrycode  235 non-null    object 
 2   population   242 non-null    float64
 3   year         242 non-null    float64
 4   maxgdp       166 non-null    float64
 5   maxhdi       164 non-null    float64
dtypes: float64(4)

In [93]:

pd.set_option('display.max_columns', 500)

In [94]:
dfworld1

,countryname,countrycode,population,year,maxgdp,maxhdi
217,Afghanistan,afg,3.629600e+07,2017.0,2025.0,0.25
339,Africa,NaN,1.244222e+09,2017.0,NaN,NaN
559,Albania,alb,2.884000e+06,2017.0,10342.0,0.59
779,Algeria,dza,4.138900e+07,2017.0,14331.0,0.45
849,American Samoa,asm,5.600000e+04,2017.0,NaN,NaN
...,...,...,...,...,...,...
48755,Western Sahara,esh,5.530000e+05,2017.0,NaN,NaN
48906,World,owid_wrl,7.547859e+09,2017.0,NaN,NaN
49126,Yemen,yem,2.783500e+07,2017.0,4811.0,0.32
49346,Zambia,zmb,1.685400e+07,2017.0,3479.0,0.31


In [95]:
dffromtobasegdp = dffromtobase.merge(dfworld1, left_on="country_from",right_on="countryname")
dffromtobasegdp = dffromtobasegdp.merge(dfworld1, left_on="country_to",right_on="countryname")
dffromtobasegdp.drop(["countryname_x","year_x","year_y","countryname_y"],axis=1,inplace=True)
dffromtobasegdp.rename(columns={"population_x":"population_from","population_y":"population_to","maxgdp_y":"maxgdp_to","maxhdi_y":"maxhdi_to","maxgdp_x":"maxgdp_from","maxhdi_x":"maxhdi_from"},inplace=True, errors="raise")
dffromtobasegdp.to_csv("N:\\Theile\\LinkedIn\\LinkedInRecruiter_dffromtobase_merged_gdp_6.csv")
dffromtobasegdp.head(30)

,country_from,country_to,number_people_who_indicated,query_time_round,linkedinusers_from,linkedinusers_to,normalized1,normalized2,countrycode_x,population_from,maxgdp_from,maxhdi_from,countrycode_y,population_to,maxgdp_to,maxhdi_to
0,Afghanistan,Central African Republic,4,2020-11-04 01:00:00,242303,21307,1.517393e-05,3.099124e-09,afg,36296000.0,2025.0,0.25,caf,4596000.0,949.0,0.16
1,Afghanistan,Central African Republic,4,2020-11-19 01:00:00,238605,20669,1.542769e-05,3.244300e-09,afg,36296000.0,2025.0,0.25,caf,4596000.0,949.0,0.16
2,Bangladesh,Central African Republic,6,2020-11-04 01:00:00,2293932,21307,2.591525e-06,7.365456e-10,bgd,159684992.0,3604.0,0.33,caf,4596000.0,949.0,0.16
3,Bangladesh,Central African Republic,6,2020-11-19 01:00:00,2338289,20669,2.543496e-06,7.448775e-10,bgd,159684992.0,3604.0,0.33,caf,4596000.0,949.0,0.16
4,Bangladesh,Central African Republic,5,2020-07-25 02:00:00,2164978,19899,2.288458e-06,5.803036e-10,bgd,159684992.0,3604.0,0.33,caf,4596000.0,949.0,0.16
5,Bangladesh,Central African Republic,6,2020-10-20 02:00:00,2305915,20598,2.578967e-06,7.579388e-10,bgd,159684992.0,3604.0,0.33,caf,4596000.0,949.0,0.16
6,Belgium,Central African Republic,5,2020-11-04 01:00:00,3706951,21307,1.341109e-06,3.165198e-10,bel,11420000.0,38766.0,0.84,caf,4596000.0,949.0,0.16
7,Belgium,Central African Republic,5,2020-11-19 01:00:00,3642140,20669,1.365073e-06,3.320962e-10,bel,11420000.0,38766.0,0.84,caf,4596000.0,949.0,0.16
8,Belgium,Central African Republic,6,2020-07-25 02:00:00,3515118,19899,1.697304e-06,5.146729e-10,bel,11420000.0,38766.0,0.84,caf,4596000.0,949.0,0.16
9,Belgium,Central African Republic,4,2020-10-20 02:00:00,3617071,20598,1.099605e-06,2.147523e-10,bel,11420000.0,38766.0,0.84,caf,4596000.0,949.0,0.16


In [17]:
dffromtobase.to_csv("LinkedInRecruiter_dffromtobase_merged_gdp.csv")

#### Total People Who want to go to one country:

In [98]:
query = """SELECT datetime(query_time, 'unixepoch', 'localtime') as "query_time2", query_country, query_info,pk_queries, response
--,datetime(min(query_time), 'unixepoch', 'localtime') as "query_time_min"
FROM queries 
WHERE query_info = 'r4'
--WHERE query_time > strftime('%s','now') -10000 and count is not null
--group by region"""
df = pd.read_sql_query(query,conn)
df.iloc[122:132,:]

,query_time2,query_country,query_info,pk_queries,response
122,2020-07-24 02:47:35,Central African Republic,r4,492,"b'{""metadata"":{""total"":1030,""searchHistoryId"":..."
123,2020-07-24 02:47:46,Rwanda,r4,496,"b'{""metadata"":{""total"":8161,""searchHistoryId"":..."
124,2020-07-24 02:47:56,Equatorial Guinea,r4,500,"b'{""metadata"":{""total"":910,""searchHistoryId"":9..."
125,2020-07-24 02:48:07,Madagascar,r4,504,"b'{""metadata"":{""total"":1387,""searchHistoryId"":..."
126,2020-07-24 02:48:17,Tanzania,r4,508,"b'{""metadata"":{""total"":9097,""searchHistoryId"":..."
127,2020-07-24 02:48:28,Paraguay,r4,512,"b'{""metadata"":{""total"":2584,""searchHistoryId"":..."
128,2020-07-24 02:48:38,El Salvador,r4,516,"b'{""metadata"":{""total"":1632,""searchHistoryId"":..."
129,2020-07-24 02:48:49,Nicaragua,r4,520,"b'{""metadata"":{""total"":1293,""searchHistoryId"":..."
130,2020-07-24 02:48:58,Guatemala,r4,524,b'CSRF check failed.'
131,2020-07-24 03:24:26,Myanmar,r4,529,"b'{""metadata"":{""total"":4526,""searchHistoryId"":..."


In [99]:
tab = []
for i, row in enumerate(df.itertuples()):
    try:
        x= json.loads(row.response)
        #print(i,row.query_country,x["metadata"]["total"])
        tab.append(x["metadata"]["total"])
    except Exception as e:
        print(i,str(e))
        tab.append(-1)

Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)
Expecting value: line 1 column 1 (char 0)


In [100]:
df["people_who_want_to_go_here"] = tab#[json.loads(row.response)]

In [103]:
del df["response"]
df.head(4)

,query_time2,query_country,query_info,pk_queries,people_who_want_to_go_here
0,2020-07-23 13:59:21,France,r4,1,163192
1,2020-07-23 14:06:33,Switzerland,r4,6,175144
2,2020-07-23 14:23:55,Russia,r4,11,31264
3,2020-07-23 14:24:25,United States,r4,15,858736


In [104]:
df.to_csv("N:\\Theile\\LinkedIn\\LinkedInRecruiter_total_number_of_people_who_want_to_go_to_each_country.csv")

In [105]:
df[df.query_country.isin(["United States", "Germany","India","China"])]

,query_time2,query_country,query_info,pk_queries,people_who_want_to_go_here
3,2020-07-23 14:24:25,United States,r4,15,858736
4,2020-07-23 14:52:44,India,r4,19,87138
23,2020-07-23 15:26:07,India,r4,95,87150
24,2020-07-23 15:26:20,United States,r4,99,858781
28,2020-07-23 15:27:10,Germany,r4,115,269911
209,2020-07-24 11:28:04,China,r4,841,81963
272,2020-10-08 14:05:32,India,r4,1096,97206
273,2020-10-08 14:06:39,United States,r4,1100,852409
277,2020-10-08 14:11:04,Germany,r4,1116,265938
284,2020-10-08 14:18:43,China,r4,1144,84265


### Add GPS-Coordinates for cool visualizations !

In [18]:
from countryinfo import CountryInfo

In [55]:
CountryInfo()

In [54]:
CountryInfo("USA").info()

{'name': 'United States',
 'altSpellings': ['US', 'USA', 'United States of America'],
 'area': 9629091,
 'borders': ['CAN', 'MEX'],
 'callingCodes': ['1'],
 'capital': 'Washington D.C.',
 'capital_latlng': [38.894986, -77.036571],
 'currencies': ['USD', 'USN', 'USS'],
 'demonym': 'American',
 'flag': '',
 'geoJSON': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'id': 'USA',
    'properties': {'name': 'United States of America'},
    'geometry': {'type': 'MultiPolygon',
     'coordinates': [[[[-155.54211, 19.08348],
        [-155.68817, 18.91619],
        [-155.93665, 19.05939],
        [-155.90806, 19.33888],
        [-156.07347, 19.70294],
        [-156.02368, 19.81422],
        [-155.85008, 19.97729],
        [-155.91907, 20.17395],
        [-155.86108, 20.26721],
        [-155.78505, 20.2487],
        [-155.40214, 20.07975],
        [-155.22452, 19.99302],
        [-155.06226, 19.8591],
        [-154.80741, 19.50871],
        [-154.83147, 19.45328],
        [-

In [30]:
CountryInfo("Afghanistan").info()#iso()

{'name': 'Afghanistan',
 'altSpellings': ['AF', 'Afġānistān'],
 'area': 652230,
 'borders': ['IRN', 'PAK', 'TKM', 'UZB', 'TJK', 'CHN'],
 'callingCodes': ['93'],
 'capital': 'Kabul',
 'capital_latlng': [34.526011, 69.177684],
 'currencies': ['AFN'],
 'demonym': 'Afghan',
 'flag': 'http://upload.wikimedia.org/wikipedia/commons/9/9a/Flag_of_Afghanistan.svg',
 'geoJSON': {'type': 'FeatureCollection',
  'features': [{'type': 'Feature',
    'id': 'AFG',
    'properties': {'name': 'Afghanistan'},
    'geometry': {'type': 'Polygon',
     'coordinates': [[[61.210817, 35.650072],
       [62.230651, 35.270664],
       [62.984662, 35.404041],
       [63.193538, 35.857166],
       [63.982896, 36.007957],
       [64.546479, 36.312073],
       [64.746105, 37.111818],
       [65.588948, 37.305217],
       [65.745631, 37.661164],
       [66.217385, 37.39379],
       [66.518607, 37.362784],
       [67.075782, 37.356144],
       [67.83, 37.144994],
       [68.135562, 37.023115],
       [68.859446, 37.344

In [34]:
CountryInfo("France").latlng()

[46, 2]

In [36]:

dffromto2 = dffromtobasegdp[(dffromtobasegdp.country_to!="gaza strip") & (dffromtobasegdp.country_from!="gaza strip")]
dffromto2

,country_from,country_to,number_people_who_indicated,query_time_round,linkedinusers_from,linkedinusers_to,normalized1,normalized2,countrycode_x,population_from,maxgdp_from,maxhdi_from,countrycode_y,population_to,maxgdp_to,maxhdi_to
0,Afghanistan,Djibouti,20,2020-07-25 02:00:00,231293,28629,0.000077,6.040756e-08,afg,36296000.0,2025.0,0.25,dji,944000.0,3064.0,0.26
1,Afghanistan,Djibouti,14,2020-10-08 02:00:00,237865,30092,0.000052,2.738259e-08,afg,36296000.0,2025.0,0.25,dji,944000.0,3064.0,0.26
2,Algeria,Djibouti,3,2020-07-25 02:00:00,1467589,28629,0.000002,2.142061e-10,dza,41389000.0,14331.0,0.45,dji,944000.0,3064.0,0.26
3,Bahrain,Djibouti,3,2020-07-25 02:00:00,284728,28629,0.000010,1.104094e-09,bhr,1494000.0,41201.0,0.57,dji,944000.0,3064.0,0.26
4,Bahrain,Djibouti,3,2020-10-08 02:00:00,290433,30092,0.000009,1.029782e-09,bhr,1494000.0,41201.0,0.57,dji,944000.0,3064.0,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9105,Colombia,Puerto Rico,105,2020-10-08 02:00:00,8909547,546467,0.000011,2.264431e-09,col,48910000.0,13326.0,0.55,pri,3164000.0,35820.0,0.64
9106,Dominican Republic,Puerto Rico,164,2020-07-25 02:00:00,670147,547201,0.000135,7.334504e-08,dom,10513000.0,14489.0,0.49,pri,3164000.0,35820.0,0.64
9107,Dominican Republic,Puerto Rico,151,2020-10-08 02:00:00,671642,546467,0.000124,6.212296e-08,dom,10513000.0,14489.0,0.49,pri,3164000.0,35820.0,0.64
9108,Mexico,Puerto Rico,78,2020-07-25 02:00:00,14681970,547201,0.000005,7.572826e-10,mex,124777000.0,16133.0,0.54,pri,3164000.0,35820.0,0.64


In [68]:
%%time

newtable = []
newcol = []
countrytable = {}

for row in dffromtobasegdp.itertuples():
    newrow = row
    try:
        if not row.country_to in countrytable:
            countrytable[row.country_to] = CountryInfo(row.country_to)#.latlng()
        newcol.append(countrytable[row.country_to])
#         print(row)
#         newrow["bu"] = CountryInfo(row.country_to)
    except Exception as e:
        print("error, ",row.country_to)
        newcol.append('')
#         newrow["bu"] = ''
    #newtable.append(newrow)
    
collat = [x.latlng()[0] for x in newcol]
collng = [x.latlng()[1] for x in newcol]
len(collat)

KeyError: 'myanmar'

In [35]:
newcol[0].latlng()

[11.5, 43]

In [45]:
%%time
print(CountryInfo("AFG").latlng())

KeyError: 'afg'

In [38]:
i=0
for c in newcol:
    i+=1
    if i
    print(c.name())

djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
djibouti
i

In [43]:
newcol[7].latlng()

[11.5, 43]

In [37]:
collat = [x.latlng()[0] for x in newcol]
collng = [x.latlng()[1] for x in newcol]
len(collat)

KeyError: 'myanmar'

In [39]:
collat[0]

NameError: name 'collat' is not defined

In [39]:
%%time
dffromto2["latlonto"] = [CountryInfo(x).latlng() for x in dffromto2["country_to"]]

KeyError: 'myanmar'

In [40]:
dffromto2

,country_from,country_to,number_people_who_indicated,query_time_round,linkedinusers_from,linkedinusers_to,normalized1,normalized2,countrycode_x,population_from,maxgdp_from,maxhdi_from,countrycode_y,population_to,maxgdp_to,maxhdi_to
0,Afghanistan,Djibouti,20,2020-07-25 02:00:00,231293,28629,0.000077,6.040756e-08,afg,36296000.0,2025.0,0.25,dji,944000.0,3064.0,0.26
1,Afghanistan,Djibouti,14,2020-10-08 02:00:00,237865,30092,0.000052,2.738259e-08,afg,36296000.0,2025.0,0.25,dji,944000.0,3064.0,0.26
2,Algeria,Djibouti,3,2020-07-25 02:00:00,1467589,28629,0.000002,2.142061e-10,dza,41389000.0,14331.0,0.45,dji,944000.0,3064.0,0.26
3,Bahrain,Djibouti,3,2020-07-25 02:00:00,284728,28629,0.000010,1.104094e-09,bhr,1494000.0,41201.0,0.57,dji,944000.0,3064.0,0.26
4,Bahrain,Djibouti,3,2020-10-08 02:00:00,290433,30092,0.000009,1.029782e-09,bhr,1494000.0,41201.0,0.57,dji,944000.0,3064.0,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9105,Colombia,Puerto Rico,105,2020-10-08 02:00:00,8909547,546467,0.000011,2.264431e-09,col,48910000.0,13326.0,0.55,pri,3164000.0,35820.0,0.64
9106,Dominican Republic,Puerto Rico,164,2020-07-25 02:00:00,670147,547201,0.000135,7.334504e-08,dom,10513000.0,14489.0,0.49,pri,3164000.0,35820.0,0.64
9107,Dominican Republic,Puerto Rico,151,2020-10-08 02:00:00,671642,546467,0.000124,6.212296e-08,dom,10513000.0,14489.0,0.49,pri,3164000.0,35820.0,0.64
9108,Mexico,Puerto Rico,78,2020-07-25 02:00:00,14681970,547201,0.000005,7.572826e-10,mex,124777000.0,16133.0,0.54,pri,3164000.0,35820.0,0.64


In [47]:
dffromto2[dffromto2.country_from=="gaza strip"]

,country_from,country_to,number_people_who_indicated,query_time_round,linkedinusers_from,linkedinusers_to,normalized1,normalized2
